Implementação da Simulação Funcional
Para a atividade, escolhi simular o Mundo dos Blocos, com 4 agentes: A, B, C e D.
Cada agente tem um objetivo específico:

A quer subir no B

B quer subir no D

C quer subir no B

D quer subir no C

Regras implementadas:
Se o agente está satisfeito (em cima do bloco desejado), ele não age.

Se não está satisfeito, tenta alcançar seu objetivo.

Se alguém impede, entra em modo agressão e força o outro a sair.

Se for agredido, entra em modo fuga e desce para a mesa.

Se não consegue fugir, também pode reagir com agressão.

Resultado da simulação:
O sistema roda por até 10 rodadas.

Os agentes agem em turnos, tentando satisfazer seus objetivos.

Devido aos conflitos entre eles, especialmente pela disputa do bloco B e o ciclo B → D → C, nem todos conseguem se manter satisfeitos ao mesmo tempo.

Ou seja, o problema não é completamente resolvido, mas a simulação mostra bem os comportamentos de agressão, fuga e dependência.

• Se satisfeito → não agir

• Se não satisfeito → tentar satisfazer

• Se impedido → agredir

• Se agredido → fugir

• Se impedir a fuga → agredir




In [ ]:
# Cada agente tem um objetivo (um outro bloco que ele quer ficar em cima)
agentes = {
    "A": {"objetivo": "B", "satisfeito": False},
    "B": {"objetivo": "D", "satisfeito": False},
    "C": {"objetivo": "B", "satisfeito": False},
    "D": {"objetivo": "C", "satisfeito": False}
}
# Nesta configuração de objetivos:
# - A quer ficar em cima do bloco B.
# - B quer ficar em cima do bloco D.
# - C quer ficar em cima do bloco B.
# - D quer ficar em cima do bloco C.
# Conflitos:
#   * A e C querem ocupar o mesmo bloco B ao mesmo tempo (conflito pelo recurso B).
#   * B quer subir em D, e D quer subir em C, o que cria dependências e conflitos indiretos.

# satisfação (alcançar o objetivo), agressão (tentar forçar outro a sair)
# fuga (ceder e sair do conflito) e dependência (esperar, caso não consiga agir no momento).


In [ ]:
# Definição do estado inicial do mundo dos blocos
posicao = {
    "A": "mesa",   # A começa na mesa (nenhum bloco abaixo dele)
    "B": "mesa",   # B começa na mesa
    "C": "mesa",   # C começa na mesa
    "D": "mesa"    # D começa na mesa
}

# No estado inicial, todos os blocos estão separados na mesa (nenhum está em cima de outro).


In [ ]:
# Função auxiliar para verificar se um bloco está livre (sem nenhum outro bloco em cima dele)
def esta_livre(bloco):
    """
    Retorna True se nenhum agente estiver diretamente em cima do 'bloco'.
    Caso contrário, retorna False.
    """
    # Se nenhum agente tem sua posição igual a esse bloco, então o bloco está livre.
    for agente, pos in posicao.items():
        if pos == bloco:
            return False  # Existe um bloco em cima do 'bloco' alvo
    return True


In [ ]:
# Função auxiliar para verificar se todos os agentes já estão satisfeitos (objetivos cumpridos)
def todos_satisfeitos():
    """
    Retorna True se todos os agentes alcançaram seus objetivos (estão satisfeitos).
    """
    for nome, dados in agentes.items():
        if not dados["satisfeito"]:
            return False
    return True

In [ ]:
# Atualiza a satisfação inicial de cada agente (no estado inicial nenhum atingiu o objetivo ainda)
for nome, dados in agentes.items():
    # Verifica se a posição atual do agente corresponde ao objetivo dele
    if posicao[nome] == dados["objetivo"]:
        agentes[nome]["satisfeito"] = True
    else:
        agentes[nome]["satisfeito"] = False

# Exibe o estado inicial antes de começar a simulação
print("Estado inicial:")
for nome in agentes:
    estado = "satisfeito" if agentes[nome]["satisfeito"] else "não satisfeito"
    print(f" - Agente {nome} está em {posicao[nome]} ({estado})")
print()

Estado inicial:
 - Agente A está em mesa (não satisfeito)
 - Agente B está em mesa (não satisfeito)
 - Agente C está em mesa (não satisfeito)
 - Agente D está em mesa (não satisfeito)



In [ ]:
# Definição da ordem de turnos dos agentes.
# Lista para determinar a ordem de ação em cada rodada.
# A cada rodada, rotacionamos a lista para alternar quem age primeiro, evitando favorecimento fixo.
ordem = ["A", "B", "C", "D"]

# Parâmetros da simulação
max_rodadas = 10  # número máximo de rodadas para evitar loop infinito caso não alcancem satisfação total

print("Iniciando a simulação passo a passo...\n")

Iniciando a simulação passo a passo...



In [ ]:
rodada = 1
while rodada <= max_rodadas:
    print(f"--- Rodada {rodada} ---")
    houve_acao = False  # indicador para saber se alguma ação ocorreu na rodada (utilizado para detectar impasse)

    # Cada agente na ordem atual tenta agir
    for nome in ordem:
        # Atualiza o estado de satisfação atual do agente antes de sua ação
        if posicao[nome] == agentes[nome]["objetivo"]:
            agentes[nome]["satisfeito"] = True
        else:
            agentes[nome]["satisfeito"] = False

        # Verifica se o agente já está satisfeito (objetivo cumprido)
        if agentes[nome]["satisfeito"]:
            # Se já estiver satisfeito, não precisa agir nesta rodada
            print(f"{nome}: Objetivo já alcançado, não precisa agir (satisfeito).")
            continue  # pula para o próximo agente

        # Se o agente não está satisfeito, ele tentará se mover para alcançar seu objetivo.
        objetivo = agentes[nome]["objetivo"]  # bloco que este agente quer ficar em cima
        # Primeiro, verifica se o agente está livre para se mover (ninguém em cima dele)
        if not esta_livre(nome):
            # Caso alguém esteja em cima deste agente, ele está bloqueado para mover.
            # Identifica quem está bloqueando (quem está em cima do agente atual)
            bloqueador = None
            for outro, pos in posicao.items():
                if pos == nome:
                    bloqueador = outro
                    break
            # Situação de dependência/agressão: o agente atual quer se mover, mas está com um peso em cima.
            # Regra de agressão: o agente de baixo (insatisfeito) força o de cima a sair.
            print(f"{nome}: Não pode se mover porque {bloqueador} está em cima dele. {nome} entra em modo agressão para remover {bloqueador}.")
            # Ação de agressão: remove o bloqueador de cima do agente atual.
            posicao[bloqueador] = "mesa"  # o bloqueador é colocado de volta na mesa (fuga do conflito)
            # Atualiza o estado de satisfação do bloqueador que foi obrigado a sair.
            if posicao[bloqueador] != agentes[bloqueador]["objetivo"]:
                agentes[bloqueador]["satisfeito"] = False
            print(f"   {bloqueador}: Foi forçado a sair de cima de {nome} e foge para a mesa (agente em fuga, agora não satisfeito).")
            houve_acao = True
            # Após essa ação de agressão, o agente atual não conseguiu ainda subir no seu alvo nesta rodada.
            # Ele tentará novamente em rodadas futuras, agora que está livre.
            continue  # passa para o próximo agente da ordem, pois gastou sua ação removendo o bloqueador

        # Se o código chegou aqui, significa que ninguém está em cima do agente (ele está livre para se mover).
        # Verifica o estado do seu objetivo (o bloco onde ele quer subir).
        alvo = objetivo  # bloco alvo em que o agente quer ficar em cima
        if not esta_livre(alvo):
            # Se o bloco alvo não está livre (tem alguém em cima dele), o agente não pode subir agora.
            # Identifica quem está ocupando o topo do bloco alvo.
            ocupante = None
            for outro, pos in posicao.items():
                if pos == alvo:
                    ocupante = outro
                    break
            # Conflito: o agente atual quer ocupar 'alvo', mas 'ocupante' está lá.
            # Regra de agressão: o agente atual (insatisfeito) força o ocupante a sair.
            print(f"{nome}: Seu alvo {alvo} está ocupado por {ocupante}. {nome} entra em modo agressão para remover {ocupante} de {alvo}.")
            # Remove o ocupante, mandando-o para a mesa (fuga do ocupante diante da agressão).
            posicao[ocupante] = "mesa"
            # Atualiza a satisfação do ocupante que caiu/foi removido do alvo.
            if posicao[ocupante] != agentes[ocupante]["objetivo"]:
                agentes[ocupante]["satisfeito"] = False
            print(f"   {ocupante}: Saiu de {alvo} e foi para a mesa (fugiu do conflito, agora não satisfeito).")
            houve_acao = True
            # O agente atual não subiu em seu alvo nesta rodada, pois gastou seu turno removendo o ocupante.
            continue  # passa para o próximo agente

        # Se chegamos aqui:
        # - O agente está livre (ninguém sobre ele).
        # - O bloco alvo também está livre no topo.
        # Isso significa que o agente pode realizar seu movimento para alcançar o objetivo.
        print(f"{nome}: Está livre e seu alvo {alvo} está livre. {nome} sobe em {alvo} para alcançar seu objetivo.")
        posicao[nome] = alvo  # atualiza a posição: coloca o agente em cima do bloco alvo
        agentes[nome]["satisfeito"] = True  # marca o agente como satisfeito, pois atingiu seu objetivo
        houve_acao = True
        print(f"   {nome}: Agora está em {alvo}. Objetivo de {nome} atingido!")

    # Fim do turno de todos os agentes nesta rodada.

    # Exibe o estado do mundo após a rodada atual
    print("Estado ao final da rodada:")
    for nome in agentes:
        estado = "satisfeito" if agentes[nome]["satisfeito"] else "não satisfeito"
        print(f" - {nome} está em {posicao[nome]} ({estado})")
    print()

    # Verifica se todos os agentes estão satisfeitos para encerrar a simulação antes do máximo de rodadas
    if todos_satisfeitos():
        print("Todos os agentes alcançaram seus objetivos! Fim da simulação.")
        break

    # Se nenhuma ação ocorreu na rodada (houve_acao False), significa que os agentes ficaram em impasse (dependência mútua possivelmente)
    if not houve_acao:
        print("Nenhuma ação foi possível nesta rodada. Os agentes podem estar em um impasse ou em estado de dependência.")
        break

    # Rotaciona a ordem dos agentes para a próxima rodada (para alternar quem começa agindo)
    primeiro = ordem.pop(0)
    ordem.append(primeiro)
    rodada += 1

# Se o loop terminou porque atingiu o número máximo de rodadas, exibi mensagem de término.
if rodada > max_rodadas:
    print("Número máximo de rodadas atingido. Fim da simulação sem que todos os resultados tenham sido atingidos")

--- Rodada 1 ---
A: Está livre e seu alvo B está livre. A sobe em B para alcançar seu objetivo.
   A: Agora está em B. Objetivo de A atingido!
B: Não pode se mover porque A está em cima dele. B entra em modo agressão para remover A.
   A: Foi forçado a sair de cima de B e foge para a mesa (agente em fuga, agora não satisfeito).
C: Está livre e seu alvo B está livre. C sobe em B para alcançar seu objetivo.
   C: Agora está em B. Objetivo de C atingido!
D: Está livre e seu alvo C está livre. D sobe em C para alcançar seu objetivo.
   D: Agora está em C. Objetivo de D atingido!
Estado ao final da rodada:
 - A está em mesa (não satisfeito)
 - B está em mesa (não satisfeito)
 - C está em B (satisfeito)
 - D está em C (satisfeito)

--- Rodada 2 ---
B: Não pode se mover porque C está em cima dele. B entra em modo agressão para remover C.
   C: Foi forçado a sair de cima de B e foge para a mesa (agente em fuga, agora não satisfeito).
C: Não pode se mover porque D está em cima dele. C entra em 